In [1]:
from __future__ import print_function, division
from collections import OrderedDict
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from torch import nn, optim, as_tensor
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

from PIL import Image
import os
import pickle
from torch.utils.data import Dataset
from torchvision import transforms as T
from torchvision.models.resnet import resnet18, resnet34, resnet50

import matplotlib.pyplot as plt
%matplotlib inline

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Matplotlib created a temporary config/cache directory at /tmp/pbs.2739780.pbsha.ib.sockeye/matplotlib-k3p15z47 because the default path (/home/jovyan/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:
# kin_data = pd.read_csv('dataset_path/inputs.csv', index_col = 0)
# output_data  = pd.read_csv('dataset_path/outputs.csv', index_col = 0)
frame_data = pd.read_csv('dataset_path/Frames.csv', index_col = 0) 
# seg_data = pd.read_csv('dataset_path/Segmentation.csv', index_col = 0)

In [3]:
# Phase_dict = {output_data['Phase'].unique()[i]:i for i in range(len(output_data['Phase'].unique()))}
# Step_dict = {output_data['Step'].unique()[i]:i for i in range(len(output_data['Step'].unique()))}
# Verb_Left_dict = {output_data['Verb_Left'].unique()[i]:i for i in range(len(output_data['Verb_Left'].unique()))}
# Verb_Right_dict = Verb_Left_dict

# print('Output Mapping Dictionary: \n')
# print('Phase Dict:', Phase_dict)
# print('Step Dict:', Step_dict)
# print('Verb Left Dict:', Verb_Left_dict)
# print('Verb Right Dict:', Verb_Right_dict)


# output_data['Phase'] = output_data['Phase'].map(Phase_dict)
# output_data['Step'] = output_data['Step'].map(Step_dict)
# output_data['Verb_Left'] = output_data['Verb_Left'].map(Verb_Left_dict)
# output_data['Verb_Right'] = output_data['Verb_Right'].map(Verb_Right_dict)

In [4]:
# ##### Input Features #####
# kin_data.head()

In [5]:
# ##### Output Features #####
# output_data.head()

In [6]:
# ##### frames #####
# frame_data.head()

In [7]:
# ##### Segmentation map #####
# seg_data.head()

In [8]:
class RandomCrop(object):
    """Randomly Crop the frames in a clip."""

    def __init__(self, output_size):
        """
            Args:
              output_size (tuple or int): Desired output size. If int, square crop
              is made.
        """
        assert isinstance(output_size, (int, tuple))
        if isinstance(output_size, int):
            self.output_size = (output_size, output_size)
        else:
            assert len(output_size) == 2
            self.output_size = output_size

    def __call__(self, clip):
        h, w = clip.size()[2:]
        new_h, new_w = self.output_size

        top = np.random.randint(0, h - new_h)
        left = np.random.randint(0, w - new_w)

        clip = clip[:, :, top : top + new_h, left : left + new_w]

        return clip


class GeneralVideoDataset(Dataset):
    """Dataset Class for Loading Video"""

    def __init__(
        self,
        seg=None,
        frame=None,
        kin=None,
        output=None,
        time_depth=None,
    ):
        """
        Args:
            clips_list_file (string): Path to the clipsList file with labels.
            root_dir (string): Directory with all the videoes.
            transform (callable, optional): Optional transform to be applied
                on a sample.
            channels: Number of channels of frames
            time_depth: Number of frames to be loaded in a sample
            x_size, y_size: Dimensions of the frames
            mean: Mean value of the training set videos over each channel
        """
        self.seg_df = seg
        self.frame_df = frame
        self.kin_df = kin
        self.output_df = output
        
        self.transform = T.Compose([T.Resize((224, 224)),
                                    T.ToTensor()])
        
#         assert len(self.seg_df) == len(self.frame_df) == len(self.kin_df) == len(self.output_df)
        # self.time_depth = time_depth

    def __len__(self):
        return len(self.frame_df)

    def read_image(self, df, idx):
        # Load the image file
        image = Image.open(df.iloc[idx]['file_path'])
        x = self.transform(image)
#         x.unsqueeze_(0)

        return x

    def __getitem__(self, idx):

        video_frame = self.read_image(self.frame_df, idx)
#         seg_map = self.read_image(self.seg_df, idx)

#         kinematic = torch.from_numpy(self.kin_df.iloc[idx].to_numpy()).float().unsqueeze_(0)
#         output = torch.from_numpy(self.output_df.iloc[idx].to_numpy())

        sample = {
            "video": video_frame,
#             "segmenation": seg_map,
#             "Kinematic": kinematic,
#             "output": output,
        }

        return sample

In [9]:
class NormalizeLayer(torch.nn.Module):
    """Standardize the channels of a batch of images by subtracting the dataset mean
      and dividing by the dataset standard deviation.

      In order to certify radii in original coordinates rather than standardized coordinates, we
      add the Gaussian noise _before_ standardizing, which is why we have standardization be the first
      layer of the classifier rather than as a part of preprocessing as is typical.
      """

    def __init__(self, means, sds):
        """
        :param means: the channel means
        :param sds: the channel standard deviations
        """
        super(NormalizeLayer, self).__init__()
        self.means = torch.tensor(means).cuda()
        self.sds = torch.tensor(sds).cuda()

    def forward(self, input: torch.tensor):
        (batch_size, num_channels, height, width) = input.shape
        means = self.means.repeat((batch_size, height, width, 1)).permute(0, 3, 1, 2)
        sds = self.sds.repeat((batch_size, height, width, 1)).permute(0, 3, 1, 2)
        return (input - means)/sds

class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x
    
    
norm_layer = NormalizeLayer(means=[0.485, 0.456, 0.406],
                            sds=[0.229, 0.224, 0.225])

resnet50_imagenet = resnet50(pretrained=False).cuda()
resnet50_img_weight_path = 'pretrained_model/checkpoint.pth.tar'
checkpoint = torch.load(resnet50_img_weight_path)
new_state_dict = OrderedDict()

for k, v in checkpoint['state_dict'].items():
    if k[:1]!=str(0):
        name = k[9:] # remove `module.`
        new_state_dict[name] = v

resnet50_imagenet.load_state_dict(new_state_dict)
resnet50_imagenet.fc = Identity()
resnet50_imagenet = torch.nn.Sequential(norm_layer, resnet50_imagenet)

for param50 in resnet50_imagenet.parameters():
    param50.requires_grad = False
    
print('resnet 50 loaded and layers freezed')

resnet 50 loaded and layers freezed


In [10]:
bs = 128
train_data = GeneralVideoDataset(frame=frame_data)
loader = DataLoader(train_data, bs, shuffle=False, num_workers=4, pin_memory=True)

In [11]:
frame_data_list = []
for batch_idx, X in enumerate(loader):
    frames = X['video']
    output_encoded = resnet50_imagenet(frames.cuda()).cpu().detach()
    frame_data_list.append(output_encoded)
    print('\rBatch: {}'.format(batch_idx+1), flush=True,  end="")

Batch: 2958

In [13]:
frame_array = torch.cat(frame_data_list, dim=0).numpy()
np.save('./dataset_path/video_frames.npy', frame_array)